In [112]:
import numpy as np
import matplotlib.pyplot as plt
import time as time

from scipy import sparse
from scipy.sparse import linalg

from matplotlib import cm
from matplotlib.ticker import LinearLocator
import logging

In [113]:
class MESH:
    def __init__(self,mat_layout,src_layout,width,n_ref,verbose):
        self.logger=logging.getLogger(__name__)
        logging.basicConfig(format="%(levelname)s: %(message)s")
        self.verbose=verbose

        if self.verbose:
            self.logger.setLevel(logging.DEBUG)
            self.logger.debug("Debugging Logger has been turned on.")
        else:
            self.logger.setLevel(logging.INFO)
            self.logger.info("Info logger has been turned on.")

        self.logger.debug("Checking input arrays' sizes")
        if not(len(mat_layout)==len(src_layout)==len(width)==len(n_ref)):

            self.logger.debug(f"Length  arrays-\n mat_layout:{len(mat_layout)},\nsrc_layout:{len(src_layout)}\nwidth:{len(width)}\nn_ref:{len(n_ref)} ")
            raise ValueError("The sizes of the inputs are not equal. Please, double check.")
        
        self.logger.debug("Arrays' sizes are consistent.") 

        self.logger.debug("Assigning arrays to the class variables")
        
        self.mat_layout=mat_layout
        self.src_layout=src_layout
        self.width=width
        self.n_ref=n_ref
        self.logger.debug("Assignment Complete!")

        self.logger.debug("Determining nzones and n_cells")
        self.nzones=len(self.n_ref)
        self.n_cells=int(sum(self.n_ref))
        self.logger.debug(f"nzones:{self.nzones}\nn_cells:{self.n_cells}")

        self.logger.debug("Checking if the arrays' values are consistent")
        for ind in range(self.nzones):
            if not (isinstance(self.mat_layout[ind],int)==isinstance(self.src_layout[ind],int)==isinstance(self.width[ind],float)==isinstance(self.n_ref[ind],int)):
                raise ValueError("The user provided arrays with type of values that were not asked for.")
            if self.width[ind]==0 or self.n_ref==0:
                raise ValueError("The width or n_ref should not have any zero element.")
        self.logger.debug("Arrays have consistent values.")
        self.logger.info("Input is consistent. I express my utmost gratitude for being sincere.")

        self.logger.debug("Creating Interface Boundary")
        self.interface_boundary=np.union1d(0,np.cumsum(self.width))
        self.logger.debug(f"Interface Boundary Array:{self.interface_boundary}")
        self.logger.debug("Determining values of edge x")
        self.x=np.zeros(1,dtype=float)
        for i in range(self.nzones):
            self.x=np.union1d(self.x,np.linspace(self.interface_boundary[i],self.interface_boundary[i+1],self.n_ref[i]+1))
        
        self.logger.debug(f"Edge x array:{self.x}")

        self.logger.debug("Assigning dx and J values")
        self.dx=np.diff(self.x)
        self.J=self.dx/2
        self.logger.debug(f"dx:{self.dx}\nJ:{self.J}")
        
        self.logger.debug("Creating cell2mat and cell2src arrays")
        self.cell2mat=np.zeros(self.n_cells,dtype=int)
        self.cell2src=np.zeros(self.n_cells,dtype=int)
        
        j=0
        for ind,value in enumerate(self.n_ref):
            for i in range(value):
                
                self.cell2mat[j]=self.mat_layout[ind]
                self.cell2src[j]=self.src_layout[ind]
                j=j+1
        self.logger.debug(f"cell2mat:{self.cell2mat}\ncell2src:{self.cell2src}")
        self.logger.info("Attribute-assignment is complete.")

    def connectivity(self):

        self.logger.debug("Inside connectivity method")

        self.gn=np.zeros((self.n_cells,2),dtype=int)
        self.gn[0,:]=[0,1]
        for k in range(1,self.n_cells):
            self.gn[k,0]=self.gn[k-1,-1]
            self.gn[k,1]=self.gn[k,0]+1
        self.logger.debug(f"Connectivity matrix:\n{self.gn}")




        
        

    



In [114]:

C=MESH([1,2,3],[4,5,6],[10.,20.,15.],[5,10,7],True)

DEBUG: Debugging Logger has been turned on.
DEBUG: Checking input arrays' sizes
DEBUG: Arrays' sizes are consistent.
DEBUG: Assigning arrays to the class variables
DEBUG: Assignment Complete!
DEBUG: Determining nzones and n_cells
DEBUG: nzones:3
n_cells:22
DEBUG: Checking if the arrays' values are consistent
DEBUG: Arrays have consistent values.
INFO: Input is consistent. I express my utmost gratitude for being sincere.
DEBUG: Creating Interface Boundary
DEBUG: Interface Boundary Array:[ 0. 10. 30. 45.]
DEBUG: Determining values of edge x
DEBUG: Edge x array:[ 0.          2.          4.          6.          8.         10.
 12.         14.         16.         18.         20.         22.
 24.         26.         28.         30.         32.14285714 34.28571429
 36.42857143 38.57142857 40.71428571 42.85714286 45.        ]
DEBUG: Assigning dx and J values
DEBUG: dx:[2.         2.         2.         2.         2.         2.
 2.         2.         2.         2.         2.         2.
 2.      

**The following is a docstringed version of my code.**

In [115]:
class MESH:
    """
    A class to represent a mesh for a numerical simulation, including layout, geometry, and material information.

    Attributes
    ----------
    mat_layout : list of int
        The material layout for each zone in the mesh. Each zone is represented by an integer.
    
    src_layout : list of int
        The source layout for each zone in the mesh. Each zone is represented by an integer.
    
    width : list of float
        The physical width of each zone in the mesh. Each zone's width is represented as a floating-point number.
    
    n_ref : list of int
        The number of cells (or refinement level) within each zone of the mesh.
    
    verbose : bool
        Flag to control verbosity. If True, detailed debugging information is logged; otherwise, only informational messages are logged.
    
    nzones : int
        Number of zones in the mesh (equal to the length of n_ref).
    
    n_cells : int
        Total number of cells in the mesh (sum of all n_ref values).
    
    interface_boundary : numpy.ndarray
        Array of interface boundaries between the zones.
    
    x : numpy.ndarray
        Array of cell edge coordinates.
    
    dx : numpy.ndarray
        Array of cell widths (differences between the edges in x).
    
    J : numpy.ndarray
        Array of half-cell widths, used for integration.
    
    cell2mat : numpy.ndarray
        Array mapping each cell to its corresponding material index.
    
    cell2src : numpy.ndarray
        Array mapping each cell to its corresponding source index.
    
    gn : numpy.ndarray
        Connectivity matrix storing the global node numbers for each cell.

    Methods
    -------
    __init__(self, mat_layout, src_layout, width, n_ref, verbose):
        Initializes the MESH class with material layout, source layout, width, and refinement level, while logging information and ensuring input consistency.

    connectivity(self):
        Builds the connectivity matrix `gn` that stores the global node numbers for each cell.
    """
    
    def __init__(self, mat_layout, src_layout, width, n_ref, verbose):
        """
        Initialize the MESH class and assign its attributes.

        Parameters
        ----------
        mat_layout : list of int
            The material layout for each zone in the mesh.
        
        src_layout : list of int
            The source layout for each zone in the mesh.
        
        width : list of float
            The physical width of each zone in the mesh.
        
        n_ref : list of int
            The number of cells (or refinement level) for each zone.
        
        verbose : bool
            If True, enables detailed debugging output for the mesh creation process.
        
        Raises
        ------
        ValueError
            If the lengths of mat_layout, src_layout, width, and n_ref are not equal, or if any width or n_ref element is zero.
        """
        self.logger = logging.getLogger(__name__)
        logging.basicConfig(format="%(levelname)s: %(message)s")
        self.verbose = verbose

        if self.verbose:
            self.logger.setLevel(logging.DEBUG)
            self.logger.debug("Debugging Logger has been turned on.")
        else:
            self.logger.setLevel(logging.INFO)
            self.logger.info("Info logger has been turned on.")

        # Checking consistency of input arrays
        self.logger.debug("Checking input arrays' sizes")
        if not (len(mat_layout) == len(src_layout) == len(width) == len(n_ref)):
            self.logger.debug(f"Array lengths:\n"
                              f"mat_layout: {len(mat_layout)}\n"
                              f"src_layout: {len(src_layout)}\n"
                              f"width: {len(width)}\n"
                              f"n_ref: {len(n_ref)}")
            raise ValueError("The sizes of the inputs are not equal. Please, double check.")
        
        self.logger.debug("Arrays' sizes are consistent.")
        
        # Assign arrays to class variables
        self.logger.debug("Assigning arrays to the class variables")
        self.mat_layout = mat_layout
        self.src_layout = src_layout
        self.width = width
        self.n_ref = n_ref
        self.logger.debug("Assignment Complete!")

        # Determine the number of zones and cells
        self.logger.debug("Determining nzones and n_cells")
        self.nzones = len(self.n_ref)
        self.n_cells = int(sum(self.n_ref))
        self.logger.debug(f"nzones: {self.nzones}\n n_cells: {self.n_cells}")

        # Check if the arrays' values are consistent
        self.logger.debug("Checking if the arrays' values are consistent")
        for ind in range(self.nzones):
            if not (isinstance(self.mat_layout[ind], int) and 
                    isinstance(self.src_layout[ind], int) and 
                    isinstance(self.width[ind], float) and 
                    isinstance(self.n_ref[ind], int)):
                raise ValueError("The user provided arrays with inconsistent types.")
            if self.width[ind] == 0 or self.n_ref[ind] == 0:
                raise ValueError("The width or n_ref should not have any zero element.")
        self.logger.debug("Arrays have consistent values.")
        self.logger.info("Input is consistent. I express my utmost gratitude for being sincere.")

        # Create interface boundary
        self.logger.debug("Creating Interface Boundary")
        self.interface_boundary = np.union1d(0, np.cumsum(self.width))
        self.logger.debug(f"Interface Boundary Array: {self.interface_boundary}")

        # Determine edge x values
        self.logger.debug("Determining values of edge x")
        self.x = np.zeros(1, dtype=float)
        for i in range(self.nzones):
            self.x = np.union1d(self.x, np.linspace(self.interface_boundary[i], 
                                                    self.interface_boundary[i+1], 
                                                    self.n_ref[i]+1))
        self.logger.debug(f"Edge x array: {self.x}")

        # Assign dx and J values
        self.logger.debug("Assigning dx and J values")
        self.dx = np.diff(self.x)
        self.J = self.dx / 2
        self.logger.debug(f"dx: {self.dx}\nJ: {self.J}")

        # Create cell2mat and cell2src arrays
        self.logger.debug("Creating cell2mat and cell2src arrays")
        self.cell2mat = np.zeros(self.n_cells, dtype=int)
        self.cell2src = np.zeros(self.n_cells, dtype=int)
        
        j = 0
        for ind, value in enumerate(self.n_ref):
            for i in range(value):
                self.cell2mat[j] = self.mat_layout[ind]
                self.cell2src[j] = self.src_layout[ind]
                j += 1
        self.logger.debug(f"cell2mat: {self.cell2mat}\ncell2src: {self.cell2src}")
        self.logger.info("Attribute-assignment is complete.")

    def connectivity(self):
        """
        Build the connectivity matrix `gn` for global node numbers of the mesh cells.

        The `gn` matrix assigns each cell a pair of global node numbers corresponding to the left and right edges of the cell.
        """
        self.logger.debug("Inside connectivity method")
        
        self.gn = np.zeros((self.n_cells, 2), dtype=int)
        self.gn[0, :] = [0, 1]
        for k in range(1, self.n_cells):
            self.gn[k, 0] = self.gn[k-1, -1]
            self.gn[k, 1] = self.gn[k, 0] + 1
        self.logger.debug(f"Connectivity matrix:\n{self.gn}")


In [116]:
C=MESH([1,2,3],[4,5,6],[10.,20.,15.],[5,10,7],True)

DEBUG: Debugging Logger has been turned on.
DEBUG: Checking input arrays' sizes
DEBUG: Arrays' sizes are consistent.
DEBUG: Assigning arrays to the class variables
DEBUG: Assignment Complete!
DEBUG: Determining nzones and n_cells
DEBUG: nzones: 3
 n_cells: 22
DEBUG: Checking if the arrays' values are consistent
DEBUG: Arrays have consistent values.
INFO: Input is consistent. I express my utmost gratitude for being sincere.
DEBUG: Creating Interface Boundary
DEBUG: Interface Boundary Array: [ 0. 10. 30. 45.]
DEBUG: Determining values of edge x
DEBUG: Edge x array: [ 0.          2.          4.          6.          8.         10.
 12.         14.         16.         18.         20.         22.
 24.         26.         28.         30.         32.14285714 34.28571429
 36.42857143 38.57142857 40.71428571 42.85714286 45.        ]
DEBUG: Assigning dx and J values
DEBUG: dx: [2.         2.         2.         2.         2.         2.
 2.         2.         2.         2.         2.         2.
 2.